In [ ]:
import requests
import time
import pandas as pd
from config import email, key
import matplotlib.pyplot as plt
import json

In [ ]:
events = ["ci37908735","nc72737985","ci37374687","nc72282711","ci15200401","ci15199681","ci10736069",
          "ci14408052","ci14383980","nc40216664","nc40204628"]

i=0

earthquakes_list = []

for event in events:
    base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
    times_url = "&starttime=1990-01-01&endtime=2019-06-15"
    magnitude_url ="&minmagnitude=5.0"
    #geometry_url = "&minlatitude=32.43&maxlatitude=42.05&minlongitude=-124.28&maxlongitude=-114.11"
    eventid_url = f"&eventid={event}"
    
    search = requests.get(base_url + times_url + magnitude_url + eventid_url)
    
    response = search.json()

    identifier = response['id']
    mag = response['properties']['mag']
    url = response['properties']['url']
    epochtime = response['properties']['time']
    four_weeks_before = epochtime - 2419200000
    four_weeks_after = epochtime + 2419200000
    event_date= time.strftime('%Y%m%d',  time.gmtime(epochtime/1000))
    month_before= time.strftime('%Y%m%d',  time.gmtime(four_weeks_before/1000))
    month_after = time.strftime('%Y%m%d',  time.gmtime(four_weeks_after/1000))
    converted_time = time.strftime('%H:%M:%S',  time.gmtime(epochtime/1000))
    timezone=response['properties']['tz']
    place = response['properties']['place']
    significance=response['properties']['sig']
    lon = response['geometry']['coordinates'][0]
    lat = response['geometry']['coordinates'][1]
    depth = response['geometry']['coordinates'][2]
    record_dict = {"Identifier":identifier, "Location":place, "Magnitude":mag, "Event Date":event_date,
                   "Month Before":month_before, "Month After":month_after, "Time":converted_time,"Time Zone":timezone, 
                   "Significance":significance, "Lat":lat, "Lon":lon, "Depth":depth, "Epoch Time":epochtime}
    earthquakes_list.append(record_dict)
        
    i += 1
          
earthquakes_df = pd.DataFrame(earthquakes_list)

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("Mexico")]

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("MX")]

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("Nevada")]

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("NV")].reset_index()

earthquakes_df = earthquakes_df.fillna(value="")

earthquakes_abbrev =earthquakes_df[['Location', 'Event Date', 'Magnitude']]

pd.set_option('display.max_rows', None)

earthquakes_df.to_csv("DataFiles/earthquake_list.csv")

earthquakes_df 

In [ ]:
earthquake = 0
record_list = []

for rows in earthquakes_df.iterrows():

    # base_url finds air quality readings by Box

    base_url = "https://aqs.epa.gov/data/api/dailyData/byBox?"

    #parameters for byBox for CRITERIA Pollutants 
    #42401="Sulfur Dioxide" 
    #42602="Nitrogen Dioxide"
    #42201="Ozone"
    #42101 = "Carbon Monoxide"

    begin = earthquakes_df.iloc[earthquake, 5]
    end = earthquakes_df.iloc[earthquake, 6]
    minlat=earthquakes_df.iloc[earthquake,10]-1
    maxlat=earthquakes_df.iloc[earthquake,10]+1
    minlon=earthquakes_df.iloc[earthquake,11]-1
    maxlon=earthquakes_df.iloc[earthquake,11]+1
    earthquake_location=earthquakes_df.iloc[earthquake,2]
    event_date =earthquakes_df.iloc[earthquake, 4]
    
    param_url = f"param=42401,42602,44201,42101,81102,88101"
    date_range_url = f"&bdate={begin}&edate={end}"
    geometry_url = f"&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}"

    url = (base_url + param_url + "&email=" + email + "&key=" + key + date_range_url + geometry_url)
    
    print(earthquake_location+" "+event_date)
    
    search = requests.get(url)
    responses = search.json()
    
    i = 0
    
    for record in responses['Data']:
        site = responses['Data'][i]['local_site_name']
        station = responses['Data'][i]['site_number']
        parameter = responses['Data'][i]['parameter']
        date = responses['Data'][i]['date_local']
        measurement = responses['Data'][i]['arithmetic_mean']
        measurement_unit = responses['Data'][i]['units_of_measure']
        event_type=responses['Data'][i]["event_type"]
        lon = responses['Data'][i]['longitude']
        lat = responses['Data'][i]['latitude']
        earthquake_id = earthquakes_df.iloc[earthquake, 1]
        record_dict = {"Earthquake ID":earthquake_id,"Parameter":parameter,"Date":date,"Station":station,"Site":site, 
                        "Lat":lat,"Lon":lon, "Measurement":measurement, "Unit":measurement_unit,"Event Type":event_type}
        record_list.append(record_dict)

        i += 1
    
    earthquake += 1

print('Seaches Completed')

In [ ]:
record_df = pd.DataFrame(record_list)
pd.set_option('display.max_rows', 100)
record_df 

In [ ]:
#pollutant2 = record df['Parameter','Unit']

In [ ]:
events = earthquakes_df['Identifier'].tolist()
pollutants = record_df['Parameter'].unique().tolist()

earthquake = 0
for event in events:

    i=0

    for pollutant in pollutants:
    
        epochtime = earthquakes_df.iloc[earthquake, 13]
        aqi_event_date = time.strftime('%Y-%m-%d',  time.gmtime(epochtime/1000))
        fancy_event_date = time.strftime('%m/%d/%Y',  time.gmtime(epochtime/1000))
        earthquake_location=earthquakes_df.iloc[earthquake,2]
        earthquake_id = earthquakes_df.iloc[earthquake, 1]
        eventname = (f'{earthquakes_df.iloc[earthquake,2]} {fancy_event_date}')
        titlename = (pollutant + ": " + eventname)
    
        graph_pollutant = record_df[record_df['Parameter']==pollutant]
        graph_pollutant = graph_pollutant[graph_pollutant['Earthquake ID']==events[earthquake]]
        graph_pollutant = graph_pollutant[['Date','Measurement']].sort_values('Date')
        graph_pollutant = graph_pollutant.groupby(['Date']).mean()
        graph_pollutant = graph_pollutant.reset_index()

        plt.plot(graph_pollutant['Date'],graph_pollutant['Measurement'])
        plt.title(titlename, fontsize=20)
        plt.xlabel("Date", fontsize = 15)
        #pollutant_unit = 
        plt.ylabel(pollutant, fontsize =15)
        plt.rcParams["figure.figsize"] = [16,6]
        plt.rcParams['xtick.labelsize']=10
        plt.rcParams['ytick.labelsize']=10
        plt.xticks(rotation=90)
        plt.axvline(x=aqi_event_date, color='r', linewidth=4)
        plt.grid()
        plt.show()
    
        i+=1
        
    earthquake += 1
    